## Introdução

## Modelo matemático
Para o modelo, será usado programação inteira mista. 

**Condições estabelecidas**

- Slots fixos de tempo
- Tarefas com duração variável (múltiplas de slot)
- Vínculos entre tarefas
- Capacidade e disponibilidade das pessoas
- Cada pessoa realiza uma tarefa por vez 
- Separação de tarefas de casa e do bebê 
- Nao sobreposição de tarefas do bebê

**Conjuntos e Parâmetros**

- Conjunto de Slots (Intervalo de tempo): $\mathcal{T} = \{0 |  1 |  \dots, S-1\}$, com $t \in \mathcal{T}$.
- Conjunto de Pessoas: $\mathcal{P}$, com $i \in \mathcal{P}$.
- Conjunto de Tarefas (Total): $\mathcal{D}$
- Subconjunto de Tarefas do Bebê: $\mathcal{D}_{B} \subset \mathcal{D}$.
- Subconjunto de Tarefas da Casa: $\mathcal{D}_{C} \subset \mathcal{D}$.
(Onde $\mathcal{D}_{B} \cup \mathcal{D}_{C} = \mathcal{D}$ e $\mathcal{D}_{B} \cap \mathcal{D}_{C} = \emptyset$)
- Ocorrências: $\mathcal{O}_j$ para a tarefa $j \in \mathcal{D}$, cada tipo de tarefa $j$ pode ocorrer várias vezes por dia.
- Capacidade (Aptidão): $c_{i,j} \in [0, 1]$, quão bem a pessoa $i$ executa a tarefa $j$.
- Duração: $d_j$ (em slots) para a tarefa $j$.
- Disponibilidade da Pessoa: $A_{i,t} \in \{0, 1\}$ slot $t$ que a pessoa $i$ está disponível.
- Disponibilidade da Tarefa: $TA_{j,t} \in \{0, 1\}$, indica se a tarefa $j$ pode ser iniciada no slot $t$.
- Limite de Carga: $L_i$, número máximo de slots que a pessoa pode ser alocada para tarefas.
- Peso do Balanceamento: $\alpha$, constante que define a penalidade para o desequilíbrio de carga (ex: 0, para considerar sem penalidade e 1000 para considerar muita penalidade).


**Variável de Decisão**

Variável binária principal:

$$x_{i,j,o,t} = \begin{cases} 
1, & \text{se a pessoa } i \text{ inicia a ocorrência } o \text{ da tarefa } j \text{ no slot } t \\
0, & \text{caso contrário}
\end{cases}$$

Variável auxiliar de balanceamento de carga:

$$\Delta \ge 0 \quad \text{(Representa a maior diferença de percentual de carga entre duas pessoas)}$$

**Função Objetivo**

Minimizar a falta de Aptidão: ANTIGA

$$\min Z = \sum_{i \in \mathcal{P}} \sum_{j \in \mathcal{D}} \sum_{o \in \mathcal{O}_j} \sum_{t \in \mathcal{T}} (1 - c_{i,j}) \cdot x_{i,j,o,t}$$

Minimizar a falta de Aptidão e o Desbalanceamento de Carga:

$$\min Z = \left( \sum_{i \in \mathcal{P}} \sum_{j \in \mathcal{D}} \sum_{o \in \mathcal{O}_j} \sum_{t \in \mathcal{T}} (1 - c_{i,j}) \cdot x_{i,j,o,t} \right) + \mathbf{\alpha \cdot \Delta}$$

**Restrições Principais**

1. Cada ocorrência de tarefa deve ser realizada exatamente uma vez:

$$\sum_{i \in \mathcal{P}} \sum_{t \in \mathcal{T}} x_{i,j,o,t} = 1 \quad \forall j \in \mathcal{D}, \forall o \in \mathcal{O}_j$$

2. Não sobreposição de tarefas por pessoa: Uma pessoa não pode realizar 2 tarefas simultaneamente (sejam elas do bebê ou da casa).

$$\sum_{j \in \mathcal{D}} \sum_{o \in \mathcal{O}_j} \sum_{t' = t}^{t + d_j} x_{i,j,o,t'} \le 1 \quad \forall i \in \mathcal{P}, \forall t \in \mathcal{T}$$

3. Não sobreposição de tarefas do bebê: No máximo uma tarefa do bebê ($\mathcal{D}_{B}$) pode estar ativa em qualquer slot $t$. Vale ressaltar que tarefas da casa ($\mathcal{D}_{C}$) não são limitadas por esta restrição.

$$\sum_{i \in \mathcal{P}} \sum_{\mathbf{j \in \mathcal{D}_{B}}} \sum_{o \in \mathcal{O}_j} \sum_{t' = \max(0, t - d_j + 1)}^{\min(t, S - d_j)} x_{i,j,o,t'} \le 1 \quad \forall t \in \mathcal{T}$$

4. Respeitar disponibilidade das pessoas: uma pessoa só pode ser alocada se estiver disponível.

$$x_{i,j,o,t} \le A_{i,t} \quad \forall i \in \mathcal{P}, \forall j \in \mathcal{D}, \forall o \in \mathcal{O}_j, \forall t \in \mathcal{T}$$

4b. Respeitar horários das Tarefas: A tarefa $j$ não pode ser iniciada no slot $t$ se o horário estiver bloqueado para ela (i.e., se $TA_{j,t} = 0$).

$$x_{i,j,o,t} \le TA_{j,t} \quad \forall i \in \mathcal{P}, \forall j \in \mathcal{D}, \forall o \in \mathcal{O}_j, \forall t \in \mathcal{T}$$

5. Precedência entre tarefas:

$$x_{i_1,j_1,o,t_1} \le \sum_{i_2 \in \mathcal{P}} \sum_{t_2 = t_1 + d_{j_1}}^{\min(S-d_{j_2}, t_1 + d_{j_1} + W)} x_{i_2,j_2,o,t_2}$$

$$\forall i_1 \in \mathcal{P}, \forall j_1 \to j_2, \forall o \in \mathcal{O}_{j_1}, \forall t_1 \in \mathcal{T}$$

6. Periodicidade (Tarefas Recorrentes):

$$\sum_{i \in \mathcal{P}} x_{i,j,o,t_1} = \sum_{i' \in \mathcal{P}} x_{i',j,o+1,t_1+P_j}$$

$$\forall j \in \mathcal{D} \text{ com periodicidade } P_j, \quad \forall o \in \{0, \dots, |\mathcal{O}_j| - 2\}, \quad \forall t_1 \in \{0, \dots, S-d_j-P_j\}$$

7. Limite de carga de trabalho:

$$\sum_{j \in \mathcal{D}} \sum_{o \in \mathcal{O}_j} \sum_{t \in \mathcal{T}} d_j \cdot x_{i,j,o,t} \le L_i \quad \forall i \in \mathcal{P}$$

8. Restrição de Balanceamento Relativo (Soft Constraint): Para garantir que todos trabalhem numa porcentagem similar do seu limite $L_i$, a variável $\Delta$ deve ser maior ou igual à diferença de ocupação entre qualquer par de pessoas $i$ e $k$.

$$\sum_{j,o,t} \frac{d_j}{L_i} x_{i,j,o,t} - \sum_{j,o,t} \frac{d_j}{L_k} x_{k,j,o,t} \le \Delta \quad \forall i, k \in \mathcal{P}, i \neq k$$



In [ ]:
print("preprocessamento")



5


In [ ]:
print("solucao")

## Discussão

A modelagem inicial não considerava o horário das tarefas, o que acarretou em tarefas sendo feitas em horários não funcionais, por exemplo, café da manhã sendo feito a tarde. Dessa forma, surgiu a necessidade de incluir essa restrição. A restrição de horário é um restrição forte e, ao colocá-la, trouxe dificuldades no que diz respeito a satisfabilidade do problema. Algumas vezes o solver não encontrou solução viável. A justificativa encontrada na análise é que conciliar o horário das tarefas com algumas precedências é muito restritivo para o problema. Por exemplo, na seguinte configuração: 

1. Configuração geral
 - Parâmetro Alpha: 1000
 - Período: 1 dia
 - Duração do Slot de Tempo: 15 minutos

2. Pessoas na rede de apoio: mãe, pai, avó, tia, tio e babá

3. Tarefas 

<div align="center">

| Tarefa | Duração (min) | Ocorrências | Categoria |
| :--- | :---: | :---: | ---: |
| Amamentar | 60 | 8 | 👶 Bebê |
| Arrotar | 30 | 8 | 👶 Bebê |
| Trocar Fralda | 30 | 8 | 👶 Bebê |
| Ninar | 30 | 8 | 👶 Bebê |
| Banho | 30 | 1 | 👶 Bebê |
| Cortar Unha | 15 | 1 | 👶 Bebê |
| Banho de Sol | 15 | 1 | 👶 Bebê |
| Lavar Louça | 15 | 3 | 🏠 Casa |
| Lavar Banheiro | 30 | 1 | 🏠 Casa |
| Tirar Lixo | 15 | 1 | 🏠 Casa |
| Aspirar Casa | 30 | 1 | 🏠 Casa |
| Lavar Roupa | 15 | 1 | 🏠 Casa |
| Passar Roupa | 30 | 1 | 🏠 Casa |
| Estender Roupa | 15 | 1 | 🏠 Casa |
| Preparar Café da Manhã | 15 | 1 | 🏠 Casa |
| Preparar Jantar | 30 | 1 | 🏠 Casa |
| Preparar Almoço | 30 | 1 | 🏠 Casa |

</div>

4. Disponibilidade de horários (todos em dia pois esses dados são apenas de 1 dia)

<div align="center">

| Pessoa | Dia | Início | Fim |
| :--- | :--- | :--- | :--- |
| **mae** | todos | 00:00 | 24:00 |
| **pai** | todos | 00:00 | 08:00 |
| **pai** | todos | 17:00 | 24:00 |
| **avo** | todos | 07:00 | 16:00 |
| **tio** | todos | 15:00 | 21:00 |
| **tia** | todos | 15:00 | 17:00 |
| **babysitter** | todos | 10:00 | 15:00 |

</div>

5. Matriz de Capacidade (Eficiência)

Valores indicam a aptidão da pessoa para realizar a tarefa (0.0 a 1.0).

<div align="center">

| Tarefa | Mae | Pai | Avo | Tio | Tia | Babysitter |
| :--- | :---: | :---: | :---: | :---: | :---: | :---: |
| amamentar | 1.0 | 0.5 | 0.5 | 0.5 | 0.5 | 0.5 |
| arrotar | 0.9 | 0.9 | 0.9 | 0.9 | 0.9 | 0.9 |
| trocar_fralda | 0.8 | 0.7 | 0.7 | 0.3 | 0.4 | 0.9 |
| ninar | 0.8 | 0.8 | 0.8 | 0.8 | 0.8 | 0.9 |
| banho | 0.9 | 0.9 | 0.9 | 0.9 | 0.9 | 0.9 |
| cortar_unha | 0.8 | 0.6 | 0.9 | 0.4 | 0.7 | 0.8 |
| banho_de_sol | 1.0 | 1.0 | 1.0 | 1.0 | 1.0 | 1.0 |
| lavar_louca | 0.7 | 0.8 | 0.6 | 0.5 | 0.7 | 0.0 |
| lavar_banheiro | 0.3 | 0.8 | 0.4 | 0.6 | 0.5 | 0.0 |
| tirar_lixo | 0.6 | 0.9 | 0.5 | 0.6 | 0.6 | 0.0 |
| aspirar_casa | 0.5 | 0.8 | 0.4 | 0.5 | 0.6 | 0.0 |
| lavar_roupa | 0.8 | 0.8 | 0.5 | 0.4 | 0.6 | 0.0 |
| passar_roupa | 0.7 | 0.6 | 0.5 | 0.3 | 0.5 | 0.0 |
| estender_roupa | 0.9 | 0.9 | 0.5 | 0.4 | 0.7 | 0.0 |
| preparar_cafe_manha | 0.8 | 0.8 | 0.7 | 0.6 | 0.7 | 0.0 |
| preparar_jantar | 0.8 | 0.8 | 0.8 | 0.5 | 0.7 | 0.0 |
| preparar_almoco | 0.8 | 0.8 | 0.9 | 0.5 | 0.7 | 0.0 |

</div>

6. Dependências de Tarefas

<div align="center">

| Tarefa Atual | Próxima Tarefa | Janela (min) |
| :--- | :--- | :---: |
| amamentar | arrotar | 0 |
| preparar_cafe_manha | lavar_louca | 300 |
| preparar_almoco | lavar_louca | 300 |
| preparar_jantar | lavar_louca | 300 |
| lavar_roupa | estender_roupa | 60 |

</div>

7. Periodicidade (Repetição)
Tarefas que devem ocorrer a cada X minutos.

<div align="center">

| Tarefa | Intervalo (min) |
| :--- | :---: |
| ninar | 180 |
| amamentar | 180 |
| trocar_fralda | 180 |

</div>

8. Limite de Carga de Trabalho
Máximo de minutos trabalhados por dia.

<div align="center">

| Pessoa | Limite (min) |
| :--- | :---: |
| mae | 1200 |
| pai | 720 |
| avo | 300 |
| tio | 120 |
| tia | 120 |
| babysitter | 300 |

</div>

9. Janelas de Execução
Intervalos de horário permitidos para tarefas específicas.

<div align="center">

| Tarefa | Início | Fim |
| :--- | :--- | :--- |
| banho | 07:00 | 21:00 |
| banho_de_sol | 07:00 | 10:00 |
| banho_de_sol | 15:00 | 17:00 |
| preparar_cafe_manha | 05:00 | 09:00 |
| preparar_almoco | 09:00 | 13:00 |
| preparar_jantar | 17:00 | 22:00 |

</div>

Nesse exemplo em expecífico, o modelo é inviável. A modificação feita para torná-lo viável foi a eliminação da dependência de café da manhã e lavar louça. É possível concluir que o único espaço possível de alocar esse lavar a louça não está na janela depois do café da manhã. Isso é possível perceber também no resultado gerado pelo solver, onde essa ocorrência de lavar a louça se encontra antes do café e está alocada para o pai. 
Além disso, um outra tentativa foi feita para 7 dias, porém o solver não executou em um intervalo de 1h, então 1 semana já é dificíl de executar. Com isso, é possível perceber que o modelo não escala muito bem. Enquanto isso, para o caso de 1 dia apenas, o modelo é resolvido em menos de 20 segundos. 

